In [29]:
from speed_trapv3.keypoints.model import SegmentationModel
from speed_trapv3.detection.model import RetinaNet
from speed_trapv3.detection.config import Config as DetConfig
from speed_trapv3.keypoints.config import Config as KeyConfig
from speed_trapv3.config import Config
from speed_trapv3.utils import slugify, get_prediction_path
from pathlib import Path
import random
from tqdm import tqdm
import imageio
from speed_trapv3.keypoints.dataset import crop_and_resize
import numpy as np
import torchvision.transforms as T
import torch
import cv2
import os


In [2]:
detection_model = RetinaNet().eval().cuda()
detection_model.load(DetConfig.trained_model_path)

In [3]:
keypoint_model = SegmentationModel().eval().cuda()
keypoint_model.load(KeyConfig.trained_model_path)

<All keys matched successfully>

In [10]:
image_transform = T.Compose([T.ToTensor()])

In [43]:
!rm -rf /code/data/datasets/temp_imgs/*

In [42]:
image_paths = list(Config.images_directory.glob("*.jpg"))
random.shuffle(image_paths)
score_threshold: float = 0.5
image_paths = image_paths[:3]
for image_path in tqdm(image_paths):
    slug = slugify(image_path)
    img = imageio.imread(image_path)
    img_h, img_w, _ = img.shape
    boxes = detection_model(img)
    boxes = boxes[boxes.scores > score_threshold].to_relative().to_tlbr()
    boxes = boxes.array[:,:4]
    keypoint_list = []
    for box in boxes:
        x1, y1, x2, y2 = (box * np.array([img_w, img_h, img_w, img_h])).astype(int)
        roi = crop_and_resize(box, img, KeyConfig.image_crop_size[0],KeyConfig.image_crop_size[1])
        # roi = np.array(roi)
        roi_w, roi_h = roi.size
        x = image_transform(roi)
        x = torch.unsqueeze(x, 0).cuda()
        keypoints = keypoint_model(x)['keypoints'][0].detach().cpu().numpy()
        keypoints =  (keypoints/ np.array([roi_w, roi_h])  * np.array([img_w, img_h])).astype(int)
        keypoint_list.append(keypoints)
        img = cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), thickness=4)
        img = cv2.circle(img, (int(keypoints[0][0]), int(keypoints[0][1])), radius=10, color=(0, 0, 255), thickness=-1) #Blue: Backtire
        img = cv2.circle(img, (int(keypoints[1][0]), int(keypoints[1][1])), radius=10, color=(255, 0, 0), thickness=-1) #Red Front tire
    save_path = '/code/data/datasets/temp_imgs'
    filename = str(slug) + ".jpg"
    cv2.imwrite(os.path.join(save_path, filename), img)
        

    # boxes.to_file(get_prediction_path(slug))

  0%|          | 0/3 [00:00<?, ?it/s]/tmp/ipykernel_71260/324182473.py:7: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imageio.imread(image_path)
100%|██████████| 3/3 [00:00<00:00,  4.00it/s]
